# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

* Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
    * excel and vensim are only supported on windows
    * vensim requires the DSS version of Vensim
    * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get it from [NetLogo 6.1.1](https://ccl.northwestern.edu/netlogo/download.shtml) 
    * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
    * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).
    

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes example scripts for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel). 

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments,
                           ema_logging,MultiprocessingEvaluator)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

from ema_workbench.analysis.plotting import lines, Density


def PredPrey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25, final_time=365, reps=1):

    #Initial values
    predators, prey, sim_time = [np.zeros((reps, int(final_time/dt)+1)) for _ in range(3)]
    
    for r in range(reps):
        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

        #Calculate the time series
        for t in range(0, sim_time.shape[1]-1):

            dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
            dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

            prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
            predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
            sim_time[r,t+1] = (t+1)*dt
    
    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

C:\ProgramData\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)
C:\ProgramData\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)


In [19]:
#Python, Netlogo, Excel

from __future__ import unicode_literals, absolute_import

if __name__ == '__main__':
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    #Netlogo
    model_netlogo = NetLogoModel('PredPrey',
                         wd="./model",
                         model_file="PredPrey.nlogo")
    model_netlogo.run_length = 100
    model_netlogo.replications = 1

    model_netlogo.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

    model_netlogo.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]
    #Python
#    model_python = Model('PredPrey', function=PredPrey)

#    model_python.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
#                      RealParameter('predation_rate', 0.0005, 0.003),
#                       RealParameter('predator_efficiency', 0.001, 0.004),
#                       RealParameter('predator_loss_rate', 0.04, 0.08)]

#    model_python.outcomes = [TimeSeriesOutcome('TIME'),
#                  TimeSeriesOutcome('predators'),
#                  TimeSeriesOutcome('prey')]
    #Excel
    model_excel = ExcelModel('PredPrey',
                         wd="./model",
                         model_file="PredPrey.xlsx")

    model_excel.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

    model_excel.outcomes  = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

    model_excel.default_sheet = "Sheet1"
    
    # perform experiments
    n = 1

    with MultiprocessingEvaluator([model_netlogo, model_excel]) as evaluator:
        results = evaluator.perform_experiments(n)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1 scenarios * 1 policies * 2 model(s) = 2 experiments
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [23]:
print(results)

experiments,outcomes = results
print(experiments)
print(outcomes)

(   predation_rate  predator_efficiency  predator_loss_rate  prey_birth_rate  \
0         0.00159             0.003797             0.07745           0.0163   
1         0.00159             0.003797             0.07745           0.0163   

  scenario policy     model  
0        2   None  PredPrey  
1        2   None  PredPrey  , {'TIME': array([[[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
         3.6475e+02, 3.6500e+02]],

       [[0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.6450e+02,
         3.6475e+02, 3.6500e+02]]]), 'predators': array([[[20.        , 20.56205255, 21.13612023, ...,  0.24524671,
          0.2433998 ,  0.24157825]],

       [[20.        , 20.56205255, 21.13612023, ...,  0.24524671,
          0.2433998 ,  0.24157825]]]), 'prey': array([[[50.        , 49.80634732, 49.60231985, ..., 12.4635738 ,
         12.51314818, 12.56292892]],

       [[50.        , 49.80634732, 49.60231985, ..., 12.4635738 ,
         12.51314818, 12.56292892]]])})
   predation_rate  preda

In [25]:
from ema_workbench.analysis import plotting, plotting_util

#for outcomes in outcomes.keys():
plotting.prepare_data(experiments, outcomes, outcomes_to_show=outcomes, filter_scalar=True)
plotting.lines(experiments, outcomes, outcomes_to_show=outcomes, 
                   density=plotting_util.Density.HIST)
plt.show()

KeyError: 'TIME'

In [ ]:
# Python

model_python = Model('PredPrey', function=PredPrey)

model_python.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

model_python.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

# perform experiments
n = 1

with MultiprocessingEvaluator(model_python) as evaluator:
    results_python = evaluator.perform_experiments(n)

In [ ]:
#Netlogo 

from __future__ import unicode_literals, absolute_import

if __name__ == '__main__':
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    model_netlogo = NetLogoModel('PredPrey',
                         wd="./model",
                         model_file="PredPrey.nlogo")
    model_netlogo.run_length = 100
    model_netlogo.replications = 1

    model_netlogo.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

    model_netlogo.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

    # perform experiments
    n = 1

    with MultiprocessingEvaluator(model_netlogo) as evaluator:
        results_netlogo = evaluator.perform_experiments(n)

In [ ]:
# Excel

if __name__ == '__main__':
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    model_excel = ExcelModel('PredPrey',
                         wd="./model",
                         model_file="PredPrey.xlsx")

    model_excel.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

    model_excel.outcomes  = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

    # name of the sheet
    model_excel.default_sheet = "Sheet1"
    
    # perform experiments
    n = 1

    with MultiprocessingEvaluator(model_excel) as evaluator:
        results_excel = evaluator.perform_experiments(n)

In [ ]:
import struct
print(struct.calcsize("P") * 8)

In [ ]:
# Vensim
from ema_workbench.connectors.vensim import VensimModel

if __name__ == "__main__":
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    # instantiate a model
    wd = './model'
    model_vensim = VensimModel('PredPrey',
                         wd="./model",
                         model_file="PredPrey.mdl")
    
    model_vensim.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency', 0.001, 0.004),
                       RealParameter('predator_loss_rate', 0.04, 0.08)]

    model_vensim.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome('predators'),
                  TimeSeriesOutcome('prey')]

    # perform experiments
    n = 1

    with MultiprocessingEvaluator(model_vensim) as evaluator:
        results_excel = evaluator.perform_experiments(n)

In [ ]:
#Visualize results
ema_workbench.analysis.plotting.lines(experiments, outcomes, outcomes_to_show=[], group_by=None, grouping_specifiers=None, density='', legend=True, titles={}, ylabels={}, experiments_to_show=None, show_envelope=False, log=False)